In [1]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


True
NVIDIA GeForce GTX 1650


In [3]:
df = pd.read_pickle('../../data/processed/balanced_df.pkl')

df['label'] = df['label'].map({'FAKE':0, 'REAL':1})

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'],df['label'], test_size=0.2, stratify=df['label'], random_state=42
)

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

max_length = 256

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length = max_length)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length = max_length)


In [5]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} | {
            "labels": torch.tensor(self.labels[idx])
        }
        
train_dataset = NewsDataset(train_encodings, list(train_labels))
val_dataset = NewsDataset(val_encodings, list(val_labels))


In [6]:
import accelerate
print(accelerate.__version__)


1.10.0


In [7]:
import sys
print(sys.executable)

c:\Users\kknah\AppData\Local\Programs\Python\Python310\python.exe


In [8]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

CUDA available: True
Device: cuda


In [9]:
# print(f"Model device: {next(model.parameters()).device}")

In [13]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)

training_args = TrainingArguments(
    output_dir='../../results',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="../../logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"   
)

print(training_args.device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda:0


In [14]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


Epoch,Training Loss,Validation Loss


In [ ]:
results = trainer.evaluate()
print(results)


In [ ]:
trainer.save_model("../../model/bert_merged")
tokenizer.save_pretrained("../../model/bert_merged")
